In [4]:
import pandas as pd
import numpy as np

In [ ]:
# Carica i due file
df_old = pd.read_csv("Data driven/Case_1_Warehouse_and_Retail_Alcohol_Sales.csv", sep=";")
df_new = pd.read_csv("Data driven/Warehouse_and_Retail_Sales_20251017.csv")

# Uniforma le colonne
def std(df):
    df = df.copy()
    df.columns = [c.strip().upper().replace(" ", "_") for c in df.columns]
    return df

old = std(df_old)
new = std(df_new)

# Scegli le chiavi di allineamento (aggiungi se vuoi: "ITEM_TYPE","SUPPLIER","ITEM_CODE")
keys = ["YEAR","MONTH"]

# Metti indice e ordina
p = old.set_index(keys).sort_index()
s = new.set_index(keys).sort_index()

# (Opzionale) considera gli 0 come "buchi" solo per queste colonne
num_cols = ["RETAIL_SALES","WAREHOUSE_SALES","RETAIL_TRANSFERS"]
for c in num_cols:
    if c in p.columns:
        p[c] = pd.to_numeric(p[c], errors="coerce")
        p[c] = p[c].replace(0, np.nan)

# Riempie i buchi del primo con i valori del secondo
filled = p.combine_first(s).reset_index()

# (Opzionale) rimetti gli NaN a 0 per esportare pulito
for c in num_cols:
    if c in filled.columns:
        filled[c] = filled[c].fillna(0)

# Salva
filled.to_csv("dataset_unificato.csv", index=False)

In [5]:
def wrangle(csv):

    df = pd.read_csv(csv)

    df = df.drop(df[df["ITEM_TYPE"] == "REF"].index)

    df = df.drop(df[df["ITEM_TYPE"] == "STR_SUPPLIES"].index)

    df = df.drop(df[df["ITEM_TYPE"] == "DUNNAGE"].index)

    df['RETAIL_SALES'] = df['RETAIL_SALES'].str.replace(',', '.')
    df["RETAIL_SALES"] = df["RETAIL_SALES"].astype("float")

    df['WAREHOUSE_SALES'] = df['WAREHOUSE_SALES'].str.replace(',', '.')
    df["WAREHOUSE_SALES"] = df["WAREHOUSE_SALES"].astype("float")

    df['RETAIL_TRANSFERS'] = df['RETAIL_TRANSFERS'].str.replace(',', '.')
    df["RETAIL_TRANSFERS"] = df["RETAIL_TRANSFERS"].astype("float")

    df['date'] = pd.to_datetime(df[['YEAR', 'MONTH']].assign(DAY=1))
    df = df.set_index('date')

    df.drop(columns=["YEAR", "MONTH"], inplace=True)

    df.dropna(inplace=True)

    df["TOTAL_SALES"] = df["WAREHOUSE_SALES"] + df["RETAIL_SALES"]

    return df